### Importera nödvändiga bibliotek
I denna cell importerar vi alla nödvändiga bibliotek för att kunna hantera bildbearbetning, realtidskamerainmatning, samt det grafiska gränssnittet.


In [2]:
import cv2
import numpy as np
import tkinter as tk

from collections import deque
from tkinter import ttk, Label
from PIL import Image, ImageTk
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

### Ladda modeller och initiera konstanter
Här laddar vi de förtränade modellerna för ålder, kön och känslodetektion samt definierar de olika kategorierna för känslor och deras respektive färger. Vi initierar också köer för att hålla förutsägelser över tid, vilket hjälper till att stabilisera resultaten.


In [37]:
# Ladda de förtränade modellerna
age_gender_model = load_model('./models/age_gender_detection_model.keras')
emotion_model = load_model('./models/improved_emotion_detection_model.keras')

# Emotion kategorier
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Färgkarta för emotioner (BGR-format)
emotion_colors = {
    'Angry': (0, 0, 255),
    'Disgust': (0, 255, 0),
    'Fear': (255, 0, 0),
    'Happy': (0, 255, 255),
    'Sad': (255, 255, 0),
    'Surprise': (255, 0, 255),
    'Neutral': (200, 200, 200)
}

# Initiera köer för att hålla förutsägelser över tid
age_queue = deque(maxlen=10)
gender_prob_queue = deque(maxlen=10)
emotion_queue = deque(maxlen=10)


### Definiera funktioner för prediktion av ålder, kön och känslor
Dessa funktioner ansvarar för att ta en ansiktsbild som indata och använda modellerna för att förutsäga ålder, kön och känslor. Resultaten används senare för att visa informationen i realtid på videoströmmen.


In [6]:
def categorize_age(predicted_age):
    if predicted_age < 6:
        return "0-5"
    elif predicted_age < 13:
        return "6-12"
    elif predicted_age < 19:
        return "13-18"
    elif predicted_age < 26:
        return "19-25"
    elif predicted_age < 36:
        return "26-35"
    elif predicted_age < 46:
        return "36-45"
    elif predicted_age < 56:
        return "46-55"
    elif predicted_age < 66:
        return "56-65"
    elif predicted_age < 76:
        return "66-75"
    elif predicted_age < 86:
        return "76-85"
    elif predicted_age < 101:
        return "86-100"
    else:
        return "100+"

def predict_age_gender(face):
    face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face_gray = cv2.resize(face_gray, (64, 64))
    face_gray = np.expand_dims(face_gray, axis=-1)
    face_gray = np.expand_dims(face_gray, axis=0)

    age_prediction, gender_prediction = age_gender_model.predict(face_gray)
    predicted_age = age_prediction[0][0]
    predicted_age_range = categorize_age(predicted_age)
    
    return predicted_age_range, gender_prediction[0][0]

def predict_emotion(face):
    face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face_gray = cv2.resize(face_gray, (48, 48))
    face_gray = face_gray.astype('float32') / 255.0
    face_gray = img_to_array(face_gray)
    face_gray = np.expand_dims(face_gray, axis=0)

    emotion_prediction = emotion_model.predict(face_gray)
    emotion_label = emotion_labels[np.argmax(emotion_prediction)]
    
    return emotion_label

def most_common(lst):
    return max(set(lst), key=lst.count)


### Definiera funktion för att rita text med kantlinje och bakgrund
Denna funktion används för att rita text på videoströmmen med både en kantlinje och en semi-transparent bakgrund. Detta hjälper till att göra texten mer läsbar oavsett vad som visas i bakgrunden.


In [8]:
def draw_text_with_background(frame, text, position, font, font_scale, text_color, bg_color, thickness=2, padding=5, alpha=0.6):
    x, y = position

    # Få textstorlek
    (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
    
    # Definiera rektangelens hörn
    top_left = (x - padding, y - text_height - padding)
    bottom_right = (x + text_width + padding, y + baseline + padding)
    
    # Rita den semi-transparenta rektangeln
    overlay = frame.copy()
    cv2.rectangle(overlay, top_left, bottom_right, bg_color, cv2.FILLED)
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    
    # Rita kantlinje
    cv2.putText(frame, text, (x, y), font, font_scale, (0, 0, 0), thickness + 2, cv2.LINE_AA)
    # Rita huvudtexten ovanpå
    cv2.putText(frame, text, (x, y), font, font_scale, text_color, thickness, cv2.LINE_AA)

### Starta webbkameran och utför prediktioner i realtid
I denna cell startar vi webbkameran, fångar ansikten i videoströmmen och gör förutsägelser om ålder, kön och känslor. Resultaten visas i realtid direkt på videoströmmen.


In [39]:
# Starta webbkameran
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Could not open webcam")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]

        # Dynamiskt effekt för rektangel runt ansikte
        scale = 1.2 
        cv2.rectangle(frame, (int(x + w * (1 - scale) / 2), int(y + h * (1 - scale) / 2)), 
                     (int(x + w * (1 + scale) / 2), int(y + h * (1 + scale) / 2)), 
                     (255, 255, 255), 2)

        # Förutsäg ålder och kön
        predicted_age_range, gender_prob = predict_age_gender(face)
        age_queue.append(int(predicted_age_range.split('-')[0]))
        gender_prob_queue.append(gender_prob)

        # Förutsäg emotion och lägg till i kön
        predicted_emotion = predict_emotion(face)
        emotion_queue.append(predicted_emotion)

        # Beräkna stabiliserade resultat
        stable_age_numeric = np.mean(age_queue)
        stable_gender_prob = np.mean(gender_prob_queue)

        # Omvandla tillbaka från numeriskt värde till kategori 
        stable_age_range = categorize_age(stable_age_numeric)
        stable_gender_str = 'Man' if stable_gender_prob < 0.3 else 'Woman'  # Justerat tröskelvärde

        # Stabilisering av emotion
        stable_emotion = most_common(list(emotion_queue))

        # Hämta färgen för den stabiliserade emotionen
        color = emotion_colors.get(stable_emotion, (255, 255, 255))  # Default till vit om emotion inte hittas

        # Justera positionerna för att undvika överlapp
        y_offset = 30  # Första texten visas 30 pixlar över ansiktets övre kant
        text_spacing = 40  # Avstånd mellan textblocken

        # Visa resultaten med justerade positioner
        draw_text_with_background(frame, f"Emotion: {stable_emotion}", (x, y - y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, (0, 0, 0), thickness=1)
        draw_text_with_background(frame, f"Age: {stable_age_range}", (x, y - y_offset - text_spacing), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, (0, 0, 0), thickness=1)
        draw_text_with_background(frame, f"Gender: {stable_gender_str}", (x, y - y_offset - 2 * text_spacing), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, (0, 0, 0), thickness=1)

    # Lägg till instruktionstexten längst ner i fönstret
    height, width, _ = frame.shape
    draw_text_with_background(frame, "Press 'Q' to quit", (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), (0, 0, 0), thickness=1)

    cv2.imshow('Age, Gender, and Emotion Detector', frame)

    # Få tangentinmatning
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━